In [1]:
import json

from pathlib import Path

from kptncook.config import settings
from kptncook.models import Recipe
from kptncook.mealie import kptncook_to_mealie

    from kptncook.mealie import MealieApiClient

In [2]:
example_path = Path.cwd().parent / "tests" / "fixtures" / "kptncook_example.json"
with example_path.open("r") as f:
    example = json.load(f)
recipe = Recipe.parse_obj(example)

In [3]:
mealie_recipe = kptncook_to_mealie(recipe)

In [4]:
# mealie_recipe.image_url

In [4]:
mealie_recipe.tags

['kptncook']

# Import via Api

In [5]:
client = MealieApiClient("http://localhost:9000/api")
client.login()

 ········


In [6]:
%%time
client.create_recipe(mealie_recipe)

CPU times: user 29.5 ms, sys: 4.17 ms, total: 33.7 ms
Wall time: 451 ms


Recipe(id=UUID('756c3a9c-739d-4598-b96d-7a1f878177a4'), user_id=UUID('899979e8-f240-4d53-8ed3-eaeac21ecd53'), group_id=UUID('71aae446-1aa5-4db7-9782-4021692cd0ce'), name='Chili sin Carne', slug='chili-sin-carne', image=None, recipe_yield=None, total_time=None, prep_time=None, cook_time=None, perform_time=None, description='', recipe_category=[], tags=[RecipeTag(name='kptncook')], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='835', fat_content=None, protein_content=None, carbohydrate_content=None, fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=False, show_nutrition=False, show_assets=False, landscape_view=False, disable_comments=True, disable_amount=True, locked=False), assets=[], notes=[RecipeNote(title='author comment', text='Hot stuff')], extras={'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'})

In [7]:
mealie_recipe.tags

['kptncook']

# Get all KptnCook Recipes

In [9]:
recipes = client.get_all_recipes()

In [10]:
recipes_with_details = []
for recipe in recipes:
    recipes_with_details.append(client.get_via_slug(recipe.slug))

In [11]:
recipes_with_details[0].extras

{'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'}

In [12]:
kc_recipes = [r for r in recipes_with_details if r.extras.get("source") == "kptncook"]

In [13]:
kc_recipes

[Recipe(id=UUID('756c3a9c-739d-4598-b96d-7a1f878177a4'), user_id=UUID('899979e8-f240-4d53-8ed3-eaeac21ecd53'), group_id=UUID('71aae446-1aa5-4db7-9782-4021692cd0ce'), name='Chili sin Carne', slug='chili-sin-carne', image=None, recipe_yield=None, total_time=None, prep_time=None, cook_time=None, perform_time=None, description='', recipe_category=[], tags=[RecipeTag(name='kptncook')], tools=[], rating=None, org_url=None, recipe_ingredient=[], date_added=None, date_updated=None, recipe_instructions=[], nutrition=Nutrition(calories='835', fat_content=None, protein_content=None, carbohydrate_content=None, fiber_content=None, sodium_content=None, sugar_content=None), settings=RecipeSettings(public=False, show_nutrition=False, show_assets=False, landscape_view=False, disable_comments=True, disable_amount=True, locked=False), assets=[], notes=[RecipeNote(title='author comment', text='Hot stuff')], extras={'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'})]

# Sync

In [14]:
from kptncook.repositories import RecipeRepository

In [15]:
repo = RecipeRepository(settings.root)

In [17]:
api_recipes = []
for repo_recipe in repo.list():
    recipe = Recipe.parse_obj(repo_recipe.data)
    mealie_recipe = kptncook_to_mealie(recipe)
    api_recipes.append(mealie_recipe)

In [18]:
len(api_recipes)

6

In [19]:
api_recipes[0].extras

{'kptncook_id': '5e5390e2740000cdf1381c64', 'source': 'kptncook'}

In [22]:
ids_in_mealie = {r.extras["kptncook_id"] for r in kc_recipes}
ids_from_api = {r.extras["kptncook_id"] for r in api_recipes}

In [25]:
ids_to_add = ids_from_api - ids_in_mealie

In [26]:
recipes_to_add = []
for recipe in api_recipes:
    if recipe.extras.get("kptncook_id") in ids_to_add:
        recipes_to_add.append(recipe)

In [27]:
len(recipes_to_add)

5

In [28]:
for recipe in recipes_to_add:
    created = client.create_recipe(recipe)